# A study of drug spending and prices in the United States

**Note: ** Please install mpld3 using pip install mpld3 for the graphs to be interactive, otherwise they can be a little convoluted

<br>
<br>

<div class="inline-block"> <img src="https://i2.wp.com/www.southernmarylandchronicle.com/wp-content/uploads/2018/01/medicare-logo.jpg?w=600" style="width: 600px;"/> </div> 


This project will study the increases in drug prices in the United States in the years 2011 - 2015. With one of the word's most expensive costs for healthcare, the public often relies on Medicare programs in order to finance their health spending. While these programs help lower acquisition costs and co-pays, they can also present a heavy burden on the budget of the country, as funds need to be redirected towards purchasing drugs. Fortunately, the government has the power, influence, and size to significantly reduce the purchasing costs of many drugs, such that the price per unit is lower than if, say, bought at a pharmacy. 

The cost structure of Medicare is of interest to me, and I wanted to take a further look into how prices rise, or remain flat for an entitiy like the U.S. government. Furthermore, I wonder if the agreements between drug companies and the government have anything to do with the number of beneficiaries of the drug. Perhaps, the most common drugs are the most stable as the sheer size and volume of purchases is all that is needed to maintain profitability. But will that mean that the government is subject to price hikes on drugs with only a few thousand, or hundred beneficiaries?


To attempt to answer this question, I will be using the Medicare Drug Spending Data, collected by Centers for Medicare & Medicaid Services, which can be found in the [CMS page](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Information-on-Prescription-Drugs/2015MedicareData.html)


## Data Report: 
As mentioned above, the data will be pulled from the CMS page. The variables in the table are the following: HCPCS Code, HCPS Description,Claim Count, Total Spending, Beneficiary Count, Total Annual Spending, Unit Count, Average Cost per Unit, and Average Beneficiary Cost Share for the years between 2011 and 2015.

### Variable Description
- `HCPCS Code:` Healthcare Common Procedure Coding System (HCPCS) code for the drug.

- `HCPCS Description:` HCPCS description of the drug.

- `Claim Count:` Number of Medicare Part B claims.  

- `Total Spending:` Aggregate drug spending for the Part B program. 

- `Beneficiary Count:` Number of Medicare Part B fee-for-service beneficiaries utilizing the drug.

- `Total Annual Spending per User:` Total Spending divided by the number of unique beneficiaries utilizing the drug (Beneficiary Count) during the benefit year.

- `Unit Count:` Total dosage units of medication billed during the calendar year (e.g. number of tablets, grams, milliliters or other units).

- `Average Cost per Unit:` Total Spending divided by the number of dosage units. 

- `Average beneficiary Cost Share:` Average amount that beneficiaries using the drug paid out of pocket during the year.

- `Annual Change in Average cost per unit (2015 only):` Annual change in average cost per unit reflects the percent change in average cost per unit between 2014 and 2015.  The average cost per unit is calculated for each year at the HCPCS level by dividing the total payment by total units and then a percentage change in unit costs between the two years is calculated. Available for drugs utilized by more than 5,000 beneficiaries in 2014 and 2015.

**Note:** For the variable Annual Change in Average Cost per unit, it is only available for the 2015 year, however I will instead calculate the difference myself based on the Average Cost Per unit across the years


### My Packages
I will be using the following packages

- `Pandas` package for importing, manipulating, and analazing data
- `Matplotlib` for plotting data
- `Mpld3` please install this using "pip install mpld3". This is important because I will be plotting 60 lines, with this I can make the graphs interactive
- `Requests`, `Zipfile` and `IO` are packages necessary for me to retrieve the data, open the zip file, and import it into a dataframe
- `Numpy` will be used to create arrays and do calculation within the array, since Pandas was giving me an error regarding creating a new column based on the values of other columns


## The Project

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt, mpld3 #Will be used to plot
import requests, zipfile, io #This helps retrieve the zip file from the page and extract it in oder to read it
import numpy as np #Will be needed for some of the array stuff

plt.rcParams['font.size'] = 12.5 #This sets a global fontsize for all labels


### Introductory Data: Putting the U.S. Federal Budget Into Perspective

Before we get started with the project, I want to highlight the importance of the questions I am asking. To do this, I have created a graph to put it all into perspective

### Retrieving the Data: Putting the U.S. Federal Budget Into Perspective

In [ ]:
#Here I am reading a CSV, getting only the rows that matter and finally putting it into a DataFrame
url = 'https://www.usgovernmentspending.com/rev/usgs_downbudget_curr.php?span=usgs302&year=2012_2017&fy=fy19&view=2&expand=&expandC=&units=b&csv=&actual='
budget = pd.read_csv(url, skiprows = 3, nrows = 10)
budget = budget[['Year','FY 2015 Outlays']]

#Here is a look at the resulting Dataframe:
print(budget)
print(budget.dtypes)

The CSV files have everything as strings, which makes it hard to perform calculations, so first we have to clean up the data a little bit, before converting to a float from a string we must remove the commas from numbers like 1,028.40

### Manipulating the data: Putting the U.S. Federal Budget Into Perspective

In [ ]:
#The CSV files have everything as strings, which makes it hard to perform calculations, 
#so first we have to clean up the data a little bit, before converting to a float from a string
#we must remove the commas from numbers like 1,028.40
for i in range (0,10):
    budget.iloc[i,1] = budget.iloc[i,1].replace(',', '')
    
budget['FY 2015 Outlays'] = budget['FY 2015 Outlays'].astype(float)
budget.rename(index=str, columns={"Year": "Sector", "FY 2015 Outlays": "2015"}, inplace = True)

#Now the data is looking nicely, and has the correct formatting, here is a look:
print(budget)
print(budget.dtypes)

### Plotting the Data: Putting the U.S. Federal Budget Into Perspective

In [ ]:
labels = budget['Sector']
explode = (0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0)  
fig1, ax1 = plt.subplots(figsize = (12,12))
ax1.pie(budget['2015'], explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=180)

ax1.axis('equal')  
plt.title('Breakdown of U.S. Federal Budget Spending',fontsize = 20)
plt.tight_layout()
plt.show()

As we can see, Health Care spending accounts for 27.9% of all Federal spending (not counting military spending, which is not included here). With such a big portion of the federal budget being spent on this, the government must surely be affected by price changes in medications that it purchases through its Medicaid programs

## Analysis of Drug Prices
With an understanding of the importance of Medicare spending to the Federal Budget of the United States, we can get started on analyzing one of the areas of spending within this sector, drug purchases for Medicare beneficiaries. If you're not familiar with the Medicare program, I just want to point out that sometimes these medication purchases require a co-pay, that is, the government pays for a portion of the drug, and the beneficiary pays for the remaining share of the cost. Since I am only interested in the U.S. Budget Data, however, I will only be looking at the U.S. consumption.
### Importing the Data

In [ ]:
r = requests.get('https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Information-on-Prescription-Drugs/Downloads/Part_B_All_Drugs_2015.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

#The filename once downloaded will become this:
filename = "Medicare_Drug_Spending_PartB_All_Drugs_YTD_2015_12_06_2016.xlsx"

Now that the data is in the directory where we are currently working, we can now put it into a dataframe.
The file has three sheets, and we're only interested in the third, named ***Data***. Furthermore, this sheet also has some rows that we need to skip in order to get to the actual table, as well as some rows with empty values. 

Below, I make sure to account for that when reading the data

In [ ]:
#The parameters sheetname and skiprows ensure we get eactly what we want, and the method dropna() ensures we only get populated data
data = pd.read_excel(filename, sheetname= 'Data', skiprows = 3)
# I choose to only work with drugs that have all the data available, so I drop all rows with NaN values
data.dropna(inplace= True)
data

### Manipulating the data
There are a lot of rows, which can make the data plotting too convoluted. Since I am particularly interested in the price changes depending on the number of users, I will take the top 30 most used drugs and the least 30 most used drugs, that way, I can compare how prices are increased depending on number of users. 

Below, I get two new Dataframes (top 30, bottom 30), using Beneficiary Count, 2011 as an indicator of number of users and then I sort the data using that column. After I get the data I want to focus on, I merge the two new dataframes to create a new one, which should have a total of 60 rows comprising just the data I want.

In [ ]:
data = data.sort_values('Beneficiary Count, 2011', ascending=False) #Sorting the data by Beneficiary Count
data_top = data.head(n=30) #Getting the most used drugs into a DataFrame
data_bottom = data.tail(n=30) #Getting the least used drugs into a DataFrame

#Merging the dataframes to create a single dataframe with the right data
frames = [data_top, data_bottom] 
drug_data = pd.concat(frames)

#Now for this specific plot, I will only be needing some columns so I create another dataframe, keeping the old one in case I
#decide to use the data
price_data = drug_data[['HCPCS Description','Average Cost Per Unit, 2011','Average Cost Per Unit, 2012',
                        'Average Cost Per Unit, 2013','Average Cost Per Unit, 2014',
                        'Average Cost Per Unit, 2015']]
price_data.columns = ['Name', '2011','2012','2013','2014','2015'] #Renaming the columns for ease of use

#Resetting the index, not actually needed but it makes the DataFrame look nicer,
#also, if someone wants to access a specific value using .loc[row, column], resetting the indexes allows for easy access
#this is preferrable to using the Name as the index, the numbers will make definitely help with retrival of specific entries
price_data.reset_index(drop=True, inplace = True) 

Finally, we have a DataFrame! It looks like this:

In [ ]:
price_data

### Plotting the Data
With the data in a DataFrame, I can now begin plotting. I was having some trouble adding new calculated columns. I was specifically getting this error:

*A value is trying to be set on a copy of a slice from a DataFrame. Try using .loc[row_indexer,col_indexer] = value instead
See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy*

Even after trying using .loc[row_indexer, col_indexer] I was having problems, so I decided to transer the data into an array.

Because some drugs are exhorbitantly more expensive than others, I decided to use the mpld3 package. This allows for two things:
- If you hover over a line, it will show you the name of the line to which it belongs
- On the bottom left of the screen there are three icons, which allow you to zoom into an area of the graph, and generally move and explore the graph

In [ ]:
price_array = price_data.as_matrix()   #converted the dataframe to an array for ease of use
new_price_array=np.delete(price_array,0,-1)  #Removing the first column, which is not needed

#Setting the parameters for the axis and the plot
year = [2011, 2012, 2013, 2014, 2015]
my_xticks = ['2011', '2012', '2013', '2014', '2015']
f, ax = plt.subplots(figsize=(10, 10))
plt.xticks(year, my_xticks)
plt.title('Absolute Dollar Value of Drugs Over Time',fontsize = 20)
ax.set_ylabel('Price in dollars (US$)',fontsize = 15)  #setting y label
ax.set_xlabel('Year',fontsize = 15)   #setting x label
plt.margins(x=0) #Making sure there is no gap between the plotted lines and the x axis
plt.margins(y=0) #Making sure there is no gap between the plotted lines and the y axis

# Plotting the drugs, I decided to go in increments of 10, instead of 30 as originally outlined in the Data Report
# because this makes it much easier to notice trends across the groups. The code below iterates through
# the array and plots the lines by groups of 10, the first 10 (orange) are the most used, 
#the bottom 10 are the least used (violet)

for i in range (0,10): #plotting first 10 as orange
    line = ax.plot(year, new_price_array[i],color='darkorange')
    #The code below connects the plotted line to its corresponding name in the array
    mpld3.plugins.connect(f, mpld3.plugins.LineLabelTooltip(line[0], label=price_array[i][0]))
for i in range (10,20): #plotting the next 10 as red 
    line = ax.plot(year, new_price_array[i],color='tomato')
    mpld3.plugins.connect(f, mpld3.plugins.LineLabelTooltip(line[0], label=price_array[i][0]))   
for i in range (20,30): #plotting the next 10 as green 
    line = ax.plot(year, new_price_array[i],color='green')
    mpld3.plugins.connect(f, mpld3.plugins.LineLabelTooltip(line[0], label=price_array[i][0]))
for i in range (30,40): #plotting the next 10 as blue 
    line = ax.plot(year, new_price_array[i],color='steelblue')
    mpld3.plugins.connect(f, mpld3.plugins.LineLabelTooltip(line[0], label=price_array[i][0]))
for i in range (40,50): #plotting the next 10 as violet 
    line = ax.plot(year, new_price_array[i],color='saddlebrown')
    mpld3.plugins.connect(f, mpld3.plugins.LineLabelTooltip(line[0], label=price_array[i][0]))    
for i in range (50,60):#plotting the next 10 as violet 
    line = ax.plot(year, new_price_array[i],color='violet')
    mpld3.plugins.connect(f, mpld3.plugins.LineLabelTooltip(line[0], label=price_array[i][0]))

mpld3.display()


**Notes on using the graph: ** This section explains how to use the interactive graph. There are three buttons [home, directional arows, zoom] (left to right) on the bottom left corner.
- **Home:** This will reset the graph to the original position and zoom
- **Arrows:** Once zoomed in, use this tool to move around the graph. Use it by selecting the tool, then dragging or 'pulling' across the graph to move
- **Zoom:** This will allow you to zoom into areas of the graph. To use it, select the tool, click and drag to create an area to zoom into. The smaller/more specific the area, the larger the zoom. You can zoom multiple times. You can also use your trackpad's multitouch feature to zoom in, currently unsure if you can use the mouse wheel, but I think so. What a great package!
- Sometimes the graph gets a little buggy with displaying names, usually you can fix this by exiting the graph with your cursor and entering touching just the line you want

### Observations:
After interacting a little and exploring the graph, there are some interesting observations.
- The least used drugs are generally the more expensive (per unit). This is noticeable because the violet lines tend to be towards the top of the graph. On average, the U.S. government pays more money per unit for these drugs.
- Similarly, the most used drugs tend to be the least costly (per unit). This is noticeable because out of the bottom 8 drugs, 6 of them are painted orange! 


### Plotting relative increases/decreases
The absolute values are interesting, and they give us important information on a specific drug's performance over time. But my research question attempts to look at the effect of price increases in drugs on the U.S. budget, therefore it is most important to use a percentage increase/decrease in a drugs price.

This will help by normalizing the data between the most expensive and cheapest drugs, and paint a clearer picture of which drugs are subject to the most volatility.

In [ ]:
#To plot this, I will create a new array with the data. Because it uses the percentage differences, it will be smaller than the
#previous by one column
percentchange = np.zeros(shape=(60,4))


for i in range(0,len(new_price_array)):
    percentchange[i][0]=(new_price_array[i][1]-new_price_array[i][0])/new_price_array[i][0]
    percentchange[i][1]=(new_price_array[i][2]-new_price_array[i][1])/new_price_array[i][1]
    percentchange[i][2]=(new_price_array[i][3]-new_price_array[i][2])/new_price_array[i][2]
    percentchange[i][3]=(new_price_array[i][4]-new_price_array[i][3])/new_price_array[i][3]

#Setting the parameters for the axis and the plot
pctchange_year = [2011,2012, 2013, 2014]
pctchange_xticks = ['2011', '2012', '2013', '2014']

pct_f, pct_ax = plt.subplots(figsize=(10, 10))
plt.xticks(pctchange_year, pctchange_xticks)
plt.title('Price Variation',fontsize = 20)
pct_ax.set_ylabel('Percent Change in Price (%)', fontsize = 15)  #setting y label
pct_ax.set_xlabel('Year', fontsize = 15)   #setting x label
plt.margins(x=0) #Making sure there is no gap between the plotted lines and the x axis
plt.margins(y=0) #Making sure there is no gap between the plotted lines and the y axis

#Plotting the drugs, again by increments, the color scheme remains the same
for i in range (0,10):
    pct_line = pct_ax.plot(pctchange_year, percentchange[i],color='darkorange')
    mpld3.plugins.connect(pct_f, mpld3.plugins.LineLabelTooltip(pct_line[0], label=price_array[i][0]))
for i in range (10,20):
    pct_line = pct_ax.plot(pctchange_year, percentchange[i],color='tomato')
    mpld3.plugins.connect(pct_f, mpld3.plugins.LineLabelTooltip(pct_line[0], label=price_array[i][0]))
for i in range (20,30):
    pct_line = pct_ax.plot(pctchange_year, percentchange[i],color='green')
    mpld3.plugins.connect(pct_f, mpld3.plugins.LineLabelTooltip(pct_line[0], label=price_array[i][0]))
for i in range (30,40):
    pct_line = pct_ax.plot(pctchange_year, percentchange[i],color='steelblue')
    mpld3.plugins.connect(pct_f, mpld3.plugins.LineLabelTooltip(pct_line[0], label=price_array[i][0]))
for i in range (40,50):
    pct_line = pct_ax.plot(pctchange_year, percentchange[i],color='saddlebrown')
    mpld3.plugins.connect(pct_f, mpld3.plugins.LineLabelTooltip(pct_line[0], label=price_array[i][0]))
for i in range (50,60):
    pct_line = pct_ax.plot(pctchange_year, percentchange[i],color='violet')
    mpld3.plugins.connect(pct_f, mpld3.plugins.LineLabelTooltip(pct_line[0], label=price_array[i][0]))

mpld3.display()

**Note on interpreting the graph:** This graph shows the amount that the price changed from year to the next. For example, the green line for the drug: "Injection,  methylprednisolone acetate, 20 mg" has a 1.2 value for the year 2011. This means that for the year 2011-2012 the drug increased by 1.2% in average price per unit

### Observations:
Now we can get to interpreting observations:
- Immediately we notice that some of the bigger  fluctuations come from two orange (most used) drugs. This means that the U.S. needs to be weary of price increases for drugs that represent a huge volume. It can easily affect the medicare spending budget and throw it off balance
- The drugs with the most fluctuation are all injections. Perhaps they saw themselves affected by a shortage? Maybe even the anti-vaccine movement
- Most drugs, however trend around the following range: -0.2 < percent change < 0.2. There doesn't seem to be a particular correlation between how many beneficiaries these drugs have and their changes in price over time. Which is good news for the Federal budget!

### Further analysis
Some of the lines with very large fluctuations have peaked my interest. Why were the fluctuations so much? Did it really come from increases in drug prices, or, since the data is Average spending per unit, from the number of beneficiaries? Fortunately, I can easily look into this data using Pandas! With a quick hover over the green, blue, and orange lines with huge fluctuations, I noted down that the drugs I am interested in are the following:
- Injection,  methylprednisolone acetate, 20 mg
- Injection, morphine sulfate, up to 10 mg
- Injection, vitamin b-12 cyanocobalamin, up to 1000 mcg
- Injection, ketorolac tromethamine, per 15 mg 

Good thing we saved the drug_data DataFrame all the way in the beginning, those extra columns will come in handy!

In [ ]:
#First, we want to set the index to the Name of the drug to make it easier to search for it.
drug_data.reset_index(drop=True, inplace = True) 

beneficiary_count = drug_data[['HCPCS Description','Beneficiary Count, 2011','Beneficiary Count, 2012',
                        'Beneficiary Count, 2013','Beneficiary Count, 2014',
                        'Beneficiary Count, 2015']]
beneficiary_count.columns = ['Name', '2011','2012','2013','2014','2015'] #Renaming the columns for ease of use

#Here I create an array with the names of the drugs I am interested in, as well as some randomly selected drugs for reference
names_of_drugs = ['Injection, methylprednisolone acetate, 20 mg','Injection, morphine sulfate, up to 10 mg'
                 ,'Injection, vitamin b-12 cyanocobalamin, up', 'Injection, ketorolac tromethamine, per 15 mg'
                  ,'Injection, dexamethasone, intravitreal implant, 0.1 mg', 'Injection, levoleucovorin calcium, 0.5 mg'
                  ,'Methotrexate sodium, 5 mg']


Now I create an array with the indexes of the drugs to do the final analysis. To do this I iterate through the names_of_drugs array, each time getting the appropriate index within the DataFrame and appending it to a new dataframe. I am sure there must be a method that returns just the index value, but I could not find it. Right now the index_of_drugs, alas, I could not find it, so I did some string slicing to each element as it came into the array